In [ ]:
#r "nuget: BserClient, 1.3.0"
#r "nuget: dotenv.net, 3.0.0"

In [ ]:
// dak.gg provides and excellent ui for eternal return black survival
// this personal api key is limited to 1 api request per second
// please do not abuse this.

In [ ]:
using System;
using System.Reflection;
using System.IO;
using BserClient;
using System.Collections.Generic;
using System.Text.Json;
using dotenv.net;
DotEnv.Load();
var envVars = DotEnv.Read();
// Set up (called once per test)
var API_KEY = envVars["BSER_API_KEY"];
var client = new BserClient.BserHttpClient(API_KEY);
// replace user id with something useful
var user_id = 1284642;

public async Task<BserClient.Types.BserUserGames> getAllGames(int user_id) {
    var games = await client.GetUserGames(user_id);
    var next = games.next;
    while(next != 0) {
        var nextGames = await client.GetUserGames(user_id, next);
        next = nextGames.next;
    }
    return games;
}
public async System.Collections.Generic.IAsyncEnumerable<BserClient.Types.BserUserGames> FetchGames(int user_id)
    {
        var games = await client.GetUserGames(user_id);
        yield return games;
        var next = games.next;
        while(next != 0) {
            var nextGames = await client.GetUserGames(user_id, next);
            next = nextGames.next;
            yield return nextGames;
        }
    }

public void WriteCSV<T>(IEnumerable<T> items, string path)
{
  Type itemType = typeof(T);
  var props = itemType.GetProperties(BindingFlags.Public | BindingFlags.Instance)
                      .OrderBy(p => p.Name);

  using (var writer = new StreamWriter(path))
  {
    writer.WriteLine(string.Join(", ", props.Select(p => p.Name)));

    foreach (var item in items)
    {
      writer.WriteLine(string.Join(", ", props.Select(p => p.GetValue(item, null))));
    }
  }
}

public static string GenerateReport<T>(List<T> items) where T : class
{
    var output = "";
    var delimiter = ",";
    var properties = typeof(T).GetProperties();
    using (var sw = new StringWriter())
    {
        var header = properties
            .Select(n => n.Name)
            .Aggregate((a, b) => a + delimiter + b);
        sw.WriteLine(header);
        foreach (var item in items)
        {
            var row = properties
                .Select(n => n.GetValue(item, null))
                .Select(n => n == null ? null : n.ToString())
                .Aggregate((a, b) => a + delimiter + b);
            sw.WriteLine(row);
        }
        output = sw.ToString();
    }
    return output;
}

List<BserClient.Types.UserGameObj>  userGames = new List<BserClient.Types.UserGameObj> {};
// https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/compiler-messages/cs8411
await foreach (var games in FetchGames(user_id))
{
    foreach (var game in games.userGames) {
        // ranked games for season 3
        if (game.matchingMode == 3 &&  game.seasonId == 3)
        {
            // solo queue only
            if (game.matchingTeamMode == 1) 
            {
                userGames.Add(game);
            }
        }
    }
    Console.WriteLine("Getting game up to " + games.next);
}

var json = JsonSerializer.Serialize(userGames);
// WriteCSV<BserClient.Types.UserGameObj>(userGames, "test.csv");
// synchronous method
// var usergames = await getAllGames(user_id);
File.WriteAllText("games.json", json);

Error: (78,58): error CS1031: Type expected